# Summing warehouse businesses by zip code with ZBP data

DT

4/20/22

This requires the zip code data to be unzipped.
ZBP data is in the same folders as the CBPs.

In [1]:
import pandas as pd
import os

In [2]:
os.listdir('../raw_data/cbp19/')

['cbp19cd.xlsx',
 'cbp19co.zip',
 'cbp19csa.zip',
 'cbp19msa.zip',
 'cbp19pr_ia_co.zip',
 'cbp19pr_ia_st.zip',
 'cbp19st.zip',
 'cbp19us.zip',
 'zbp19detail.txt',
 'zbp19detail.zip',
 'zbp19totals.zip']

In [3]:
%%time
zbp19 = pd.read_csv('../raw_data/cbp19/zbp19detail.txt', encoding = 'ISO-8859-1')
zbp19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881311 entries, 0 to 2881310
Data columns (total 16 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   zip       int64 
 1   name      object
 2   naics     object
 3   est       int64 
 4   n<5       object
 5   n5_9      object
 6   n10_19    object
 7   n20_49    object
 8   n50_99    object
 9   n100_249  object
 10  n250_499  object
 11  n500_999  object
 12  n1000     object
 13  city      object
 14  stabbr    object
 15  cty_name  object
dtypes: int64(2), object(14)
memory usage: 351.7+ MB
Wall time: 4.28 s


In [4]:
zbp19['zip'].value_counts()

33166    830
10001    816
92705    760
11101    746
80112    729
        ... 
72812      1
28272      1
72776      1
72769      1
501        1
Name: zip, Length: 35129, dtype: int64

In [5]:
zbp19['naics'].value_counts()[:20]

------    35129
81----    22504
23----    22427
44----    21916
238///    19475
72----    19076
813///    18897
722///    18117
62----    17955
541///    17748
54----    17748
72251/    16735
7225//    16735
56----    16539
813110    16416
8131//    16416
81311/    16416
48----    16118
561///    15812
31----    15775
Name: naics, dtype: int64

There are 35129 total businesses.
The big ones:
- 81----    is Other Services.
- 23----    is construction. (This could be useful when thinking about warehouses?? How do construction business work in terms of establishments vs sites?)
- 44----	Retail Trade
- 238///	Specialty Trade Contractors

More in the NAICS notebook. The ones we're focused on are the warehouses.

Warehouse NAICS:
Our codes are `493///` for Warehousing and Storage.
`4931//` is the only subset of the above. (like taxonomy -- _sapiens_ is the only living species in _homo_)
- `49311/`, `493110` General
- `49312/`, `493120` Refrigerated
- `49313/`, `493130` Farm Products
- `49319/`, `493190` Other


It will be helpful to first just extract only the rows with zips in cali.

4/21/22

made a couple dictionaries to translate tracts to zips and vice versa.

In [6]:
import json

dicts_filepath = '../CA_zip_tract_translation/'

tract_to_zip_dict_filepath = dicts_filepath + 'tract_to_zip.json'
zip_to_tract_dict_filepath = dicts_filepath + 'zip_to_tract.json'

with open (tract_to_zip_dict_filepath, 'r') as my_ttz_file:
    tract_to_zip = json.load(my_ttz_file)

with open (zip_to_tract_dict_filepath, 'r') as my_ztt_file:
    zip_to_tract = json.load(my_ztt_file)

In [7]:
zip_to_tract['90210']

[6037700801, 6037700700, 6037141700, 6037261101, 6037700600, 6037261102]

In [8]:
tract_to_zip['6037700801']

90210

I'd like a DataFrame with observations like the following...


| index | census tract |  zip code | original CAES number | total business type 1 | total business type 2 | ... |
| --- | --- | --- | --- | --- | --- | --- |

making sure not to use census tract or zip as indices --- there will be multiple ones from each CAES set.

In [9]:
all_cali_zips = set(zip_to_tract.keys())
all_cali_zips

{'91977',
 '93030',
 '90043',
 '91104',
 '95925',
 '95827',
 '93442',
 '91776',
 '93727',
 '95203',
 '94704',
 '95206',
 '96142',
 '92310',
 '95691',
 '91765',
 '94102',
 '91505',
 '93630',
 '91020',
 '95616',
 '94534',
 '95728',
 '90022',
 '92103',
 '95864',
 '92673',
 '92113',
 '91303',
 '92808',
 '94508',
 '90015',
 '92140',
 '93225',
 '93517',
 '96088',
 '93907',
 '93305',
 '95062',
 '93657',
 '93035',
 '92843',
 '32',
 '94949',
 '92315',
 '95664',
 '91390',
 '95117',
 '90011',
 '94535',
 '94024',
 '92883',
 '95525',
 '95973',
 '91911',
 '93203',
 '95301',
 '90265',
 '94114',
 '94103',
 '95003',
 '92553',
 '93955',
 '92106',
 '90815',
 '94571',
 '90254',
 '95757',
 '95210',
 '95060',
 '95602',
 '92086',
 '95688',
 '90404',
 '90010',
 '90717',
 '94572',
 '94801',
 '95823',
 '90262',
 '93633',
 '94301',
 '93656',
 '90067',
 '91307',
 '95641',
 '91306',
 '90017',
 '93622',
 '95357',
 '90270',
 '95988',
 '91773',
 '94519',
 '91107',
 '95690',
 '95128',
 '95814',
 '95037',
 '92136',
 '9

In [10]:
#there are almost 300000 observations from california in zbp19.

#make a table of bools to mask the non-calis
zbp19_cali_truths = zbp19['zip'].apply(lambda zip_code: str(zip_code) in all_cali_zips)
zbp19_cali_truths.sum()

299337

In [11]:
#mask the non-calis with that truth table.
zbp19_cali = zbp19[zbp19_cali_truths]
zbp19_cali

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443695,90001,"LOS ANGELES, CA",------,613,316,111,96,73,4,12,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443696,90001,"LOS ANGELES, CA",23----,28,13,8,6,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443697,90001,"LOS ANGELES, CA",236///,6,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443698,90001,"LOS ANGELES, CA",2361//,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443699,90001,"LOS ANGELES, CA",23611/,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749280,96161,"TRUCKEE, CA",81331/,8,5,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749281,96161,"TRUCKEE, CA",813312,7,4,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749282,96161,"TRUCKEE, CA",8139//,11,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749283,96161,"TRUCKEE, CA",81399/,9,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA


In [12]:
zbp19_cali.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299337 entries, 2443695 to 2749284
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   zip       299337 non-null  int64 
 1   name      299337 non-null  object
 2   naics     299337 non-null  object
 3   est       299337 non-null  int64 
 4   n<5       299337 non-null  object
 5   n5_9      299337 non-null  object
 6   n10_19    299337 non-null  object
 7   n20_49    299337 non-null  object
 8   n50_99    299337 non-null  object
 9   n100_249  299337 non-null  object
 10  n250_499  299337 non-null  object
 11  n500_999  299337 non-null  object
 12  n1000     299337 non-null  object
 13  city      299337 non-null  object
 14  stabbr    299337 non-null  object
 15  cty_name  299337 non-null  object
dtypes: int64(2), object(14)
memory usage: 38.8+ MB


In [13]:
# look at construction. There are some with '-' and some with '/' as their placeholders.
# I'm not sure there's a difference between the two symbols.
zbp19_cali[zbp19_cali['naics'] == '23----']

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443696,90001,"LOS ANGELES, CA",23----,28,13,8,6,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443959,90002,"LOS ANGELES, CA",23----,21,17,N,3,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444047,90003,"LOS ANGELES, CA",23----,28,22,4,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444196,90004,"LOS ANGELES, CA",23----,50,34,12,3,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444565,90005,"LOS ANGELES, CA",23----,34,27,5,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748356,96142,"TAHOMA, CA",23----,14,11,N,N,N,N,N,N,N,N,TAHOMA,CA,EL DORADO
2748368,96143,"KINGS BEACH, CA",23----,29,23,4,N,N,N,N,N,N,N,KINGS BEACH,CA,PLACER
2748421,96145,"TAHOE CITY, CA",23----,60,40,14,5,N,N,N,N,N,N,TAHOE CITY,CA,PLACER
2748648,96150,"SOUTH LAKE TAHOE, CA",23----,154,124,20,8,N,N,N,N,N,N,SOUTH LAKE TAHOE,CA,EL DORADO


In [14]:
#221 "summary" rows with ALL warehouses in each zip. 
zbp19_cali[zbp19_cali['naics'] == '493///']

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443850,90001,"LOS ANGELES, CA",493///,3,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2449655,90021,"LOS ANGELES, CA",493///,9,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2450345,90023,"LOS ANGELES, CA",493///,14,3,N,N,4,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2455713,90040,"LOS ANGELES, CA",493///,16,5,3,3,N,3,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2456997,90045,"LOS ANGELES, CA",493///,6,N,3,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739721,95834,"SACRAMENTO, CA",493///,4,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2740236,95838,"SACRAMENTO, CA",493///,3,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2742450,95945,"GRASS VALLEY, CA",493///,3,N,N,N,N,N,N,N,N,N,GRASS VALLEY,CA,NEVADA
2744460,95987,"WILLIAMS, CA",493///,7,5,N,N,N,N,N,N,N,N,WILLIAMS,CA,COLUSA


In [15]:
#make a dict for converting warehouse types to codes.
warehouse_dict = {
    'warehouses':'493///',
    'gen/':'49311/',
    'gen':'493110',
    'cold':'49312/',
    'farm/':'49313/',
    'farm':'493130',
    'other/':'49319/',
    'other':'493190'
}

In [16]:
#try it out.
df2 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['gen'] ]

In [17]:
# the general code has the same statistics as the more specific code.

df1 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['gen/']]

In [18]:
#just checking if dividing by 'gen' type is the same as dividing by 'gen/' type.
df1.drop(columns = 'naics').values == df2.drop(columns = 'naics').values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [19]:
sum(_)

array([174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174,
       174, 174])

In [20]:
len(df1)

174

In [21]:
# ^ They are the same. ^

In [22]:
df1 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['cold']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 2449659 to 2715469
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zip       20 non-null     int64 
 1   name      20 non-null     object
 2   naics     20 non-null     object
 3   est       20 non-null     int64 
 4   n<5       20 non-null     object
 5   n5_9      20 non-null     object
 6   n10_19    20 non-null     object
 7   n20_49    20 non-null     object
 8   n50_99    20 non-null     object
 9   n100_249  20 non-null     object
 10  n250_499  20 non-null     object
 11  n500_999  20 non-null     object
 12  n1000     20 non-null     object
 13  city      20 non-null     object
 14  stabbr    20 non-null     object
 15  cty_name  20 non-null     object
dtypes: int64(2), object(14)
memory usage: 2.7+ KB


In [23]:
df1

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2449659,90021,"LOS ANGELES, CA",49312/,4,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2450349,90023,"LOS ANGELES, CA",49312/,6,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2459476,90058,"LOS ANGELES, CA",49312/,17,5,N,N,7,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2481757,90640,"MONTEBELLO, CA",49312/,3,N,N,N,N,N,N,N,N,N,MONTEBELLO,CA,LOS ANGELES
2487475,90744,"WILMINGTON, CA",49312/,3,N,N,N,N,N,N,N,N,N,WILMINGTON,CA,LOS ANGELES
2519220,91706,"BALDWIN PARK, CA",49312/,3,N,N,N,N,N,N,N,N,N,BALDWIN PARK,CA,LOS ANGELES
2521921,91730,"RANCHO CUCAMONGA, CA",49312/,3,N,N,N,N,N,N,N,N,N,RANCHO CUCAMONGA,CA,SAN BERNARDINO
2527528,91761,"ONTARIO, CA",49312/,6,N,N,3,N,N,N,N,N,N,ONTARIO,CA,SAN BERNARDINO
2567685,92335,"FONTANA, CA",49312/,3,N,N,N,N,N,N,N,N,N,FONTANA,CA,SAN BERNARDINO
2575519,92509,"RIVERSIDE, CA",49312/,5,N,N,N,N,N,N,N,N,N,RIVERSIDE,CA,RIVERSIDE


20 entries in the refrigerated class.

In [24]:
zbp19_cali[zbp19_cali['naics'] == warehouse_dict['farm']]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2617853,93212,"CORCORAN, CA",493130,3,N,N,N,N,N,N,N,N,N,CORCORAN,CA,KINGS
2639014,93725,"FRESNO, CA",493130,3,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO


Only two in the farm type.

In [25]:
zbp19_cali[zbp19_cali['naics'] == warehouse_dict['other']]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2459479,90058,"LOS ANGELES, CA",493190,5,N,3,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2482512,90660,"PICO RIVERA, CA",493190,6,N,N,N,N,N,N,N,N,N,PICO RIVERA,CA,LOS ANGELES
2483086,90670,"SANTA FE SPRINGS, CA",493190,4,N,N,N,N,N,N,N,N,N,SANTA FE SPRINGS,CA,LOS ANGELES
2484013,90703,"CERRITOS, CA",493190,3,N,N,N,N,N,N,N,N,N,CERRITOS,CA,LOS ANGELES
2487742,90745,"CARSON, CA",493190,3,N,N,N,N,N,N,N,N,N,CARSON,CA,LOS ANGELES
2491144,90813,"LONG BEACH, CA",493190,3,N,N,N,N,N,N,N,N,N,LONG BEACH,CA,LOS ANGELES
2525722,91748,"ROWLAND HEIGHTS, CA",493190,5,N,N,N,N,N,N,N,N,N,ROWLAND HEIGHTS,CA,LOS ANGELES
2527531,91761,"ONTARIO, CA",493190,6,N,N,N,N,N,N,N,N,N,ONTARIO,CA,SAN BERNARDINO
2529444,91766,"POMONA, CA",493190,3,N,N,N,N,N,N,N,N,N,POMONA,CA,LOS ANGELES
2537631,91950,"NATIONAL CITY, CA",493190,3,N,N,N,N,N,N,N,N,N,NATIONAL CITY,CA,SAN DIEGO


# counting and combining

PLAN:
- get a list of all unique zips
    - that's easy. I already have it from the zip to tract dictionary.
- create a new dataframe
- for each zip,
    - create an observation for each zip / year with est total, est gen, est farm, est cold, est other.
    - boolean mask out all the observations with that zip
    - for each observation, add est to gen/farm/cold/other total
    

In [65]:
#make a new DF with the desired columns
columns = ['zip', 'year', 'est total', 'est gen', 'est cold', 'est farm', 'est other']
zip_df_19 = pd.DataFrame(columns = columns)

In [66]:
#give the DF the right . We'll do this again for each of the other years.
zip_df_19['zip'] = zip_to_tract.keys()
zip_df_19['year'] = 2019
zip_df_19

,zip,year,est total,est gen,est cold,est farm,est other
0,93706,2019,NaN,NaN,NaN,NaN,NaN
1,91761,2019,NaN,NaN,NaN,NaN,NaN
2,95203,2019,NaN,NaN,NaN,NaN,NaN
3,93725,2019,NaN,NaN,NaN,NaN,NaN
4,90023,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1350,95389,2019,NaN,NaN,NaN,NaN,NaN
1351,94964,2019,NaN,NaN,NaN,NaN,NaN
1352,95720,2019,NaN,NaN,NaN,NaN,NaN
1353,57,2019,NaN,NaN,NaN,NaN,NaN


In [86]:
#make a dictionary to make searching zbp19_cali easier.
business_filter_dict = {
    'total':'493///',
    'gen':'493110',
    'cold':'49312/',
    'farm':'493130',
    'other':'493190' 
}

In [87]:
#just testing out how entries of a dict are spat out.
for business_type in business_filter_dict:
    print(business_type)

total
gen
cold
farm
other


In [88]:
#the result was just the list of keys...
for key in business_filter_dict.keys():
    print(key)

total
gen
cold
farm
other


In [89]:
# ... not the values.
for value in business_filter_dict.values():
    print(value)

493///
493110
49312/
493130
493190


In [99]:
business_filter_dict

{'total': '493///',
 'gen': '493110',
 'cold': '49312/',
 'farm': '493130',
 'other': '493190'}

In [100]:
unique_zips[0]

93706

In [105]:
zip_filter = zbp19['zip'].apply(lambda zipcode: zipcode == 93706)

In [106]:
zip_filter.sum()

274

In [107]:
this_zip_df_19 = zbp19[zip_filter]

In [108]:
this_zip_df_19

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2636654,93706,"FRESNO, CA",------,572,227,109,96,82,36,16,N,N,3,FRESNO,CA,FRESNO
2636655,93706,"FRESNO, CA",11----,5,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636656,93706,"FRESNO, CA",115///,5,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636657,93706,"FRESNO, CA",1151//,4,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636658,93706,"FRESNO, CA",11511/,4,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636923,93706,"FRESNO, CA",812220,3,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636924,93706,"FRESNO, CA",813///,21,14,3,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636925,93706,"FRESNO, CA",8131//,15,11,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO
2636926,93706,"FRESNO, CA",81311/,15,11,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO


In [112]:
this_business_filter = this_zip_df_19['naics'].apply(lambda naic: naic == business_filter_dict[business_type])

In [122]:
this_zip_business_df_19 = this_zip_df_19[this_business_filter]
this_zip_business_df_19

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2636807,93706,"FRESNO, CA",493///,6,3,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO


In [119]:
zip_df_19.loc[zip_df_19['zip'] == str(93706),'est ' + 'gen'] =50

In [120]:
zip_df_19

,zip,year,est total,est gen,est cold,est farm,est other
0,93706,2019,0,50,0,0,0
1,91761,2019,0,0,0,0,0
2,95203,2019,0,0,0,0,0
3,93725,2019,0,0,0,0,0
4,90023,2019,0,0,0,0,0
...,...,...,...,...,...,...,...
1350,95389,2019,0,0,0,0,0
1351,94964,2019,0,0,0,0,0
1352,95720,2019,0,0,0,0,0
1353,57,2019,0,0,0,0,0


In [137]:

%%time
##is there a better way to do this? probably. But this way works. I think.
counter = 0
#for each unique zip code,
for this_zip_code in unique_zips:
    counter += 1
    if counter%100 == 0:
        print(f'counter == {counter}')
# get a dataframe from zbp19_cali or zbp19 with only that zip code
    zip_filter = zbp19_cali['zip'].apply(lambda zipcode: zipcode == this_zip_code)
    this_zip_df_19 = zbp19_cali[zip_filter]
# for each business type,
    for business_type in business_filter_dict:        
# add up all the entries in the est column for that type,  and put that in zip_df_19 for that zip code.
        business_filter = this_zip_df_19['naics'].apply(lambda naic: naic == business_filter_dict[business_type])
        this_zip_business_df_19 = this_zip_df_19[business_filter]
        #it gives an error if there are no businesses of that type in the zip code. avoid this
        if len(this_zip_business_df_19) > 0:
            zip_df_19.loc[zip_df_19['zip'] == str(this_zip_code),'est ' + business_type] = this_zip_business_df_19['est'].sum()
        


counter == 100
counter == 200
counter == 300
counter == 400
counter == 500
counter == 600
counter == 700
counter == 800
counter == 900
counter == 1000
counter == 1100
counter == 1200
counter == 1300
Wall time: 1min 25s


In [139]:
zip_df_19

,zip,year,est total,est gen,est cold,est farm,est other
0,93706,2019,6,3,0,0,0
1,91761,2019,97,84,6,0,6
2,95203,2019,3,0,0,0,0
3,93725,2019,20,9,3,3,5
4,90023,2019,14,7,6,0,0
...,...,...,...,...,...,...,...
1350,95389,2019,0,0,0,0,0
1351,94964,2019,0,0,0,0,0
1352,95720,2019,0,0,0,0,0
1353,57,2019,0,0,0,0,0


In [ ]:
#okay. I think it worked. save it while you can.
zip_df_19